# Document Classification with Naive Bayes - Lab

## Introduction

In this lesson, you'll practice implementing the Naive Bayes algorithm on your own.

## Objectives

In this lab you will:  

* Implement document classification using Naive Bayes

## Import the dataset

To start, import the dataset stored in the text file `'SMSSpamCollection'`.

In [74]:
import pandas as pd
df = pd.read_csv('SMSSpamCollection', sep='\t', names=['label', 'text'])
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Account for class imbalance

To help your algorithm perform more accurately, subset the dataset so that the two classes are of equal size. To do this, keep all of the instances of the minority class (spam) and subset examples of the majority class (ham) to an equal number of examples.

In [75]:
spam = df[df['label']=='spam']
ham = df[df['label']=='ham']
ham_sam = ham.sample(len(spam),random_state= 17)
df_balanced = pd.concat([spam,ham_sam],axis=0)

## Train-test split

Now implement a train-test split on the dataset: 

In [76]:
# Your code here
from sklearn.model_selection import train_test_split
X = df_balanced.drop('label',axis=1)
y = df_balanced['label']
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=17)
train_df = pd.concat([y_train,X_train], axis=1)
test_df = pd.concat([y_test,X_test], axis=1)

## Create the word frequency dictionary for each class

Create a word frequency dictionary for each class: 

In [77]:
class_word_freq = {} 
classes = train_df['label'].unique()
for class_ in classes:
    temp_df = train_df[train_df['label'] == class_]
    bag = {}
    for row in temp_df.index:
        doc = temp_df['text'][row]
        for word in doc.split():
            bag[word] = bag.get(word, 0) + 1
    class_word_freq[class_] = bag

## Count the total corpus words
Calculate V, the total number of words in the corpus: 

In [78]:
vocabulary = set()
for text in train_df['text']:
    for word in text.split():
        vocabulary.add(word)
V = len(vocabulary)
V

5955

## Create a bag of words function

Before implementing the entire Naive Bayes algorithm, create a helper function `bag_it()` to create a bag of words representation from a document's text.

In [79]:
def bag_it(doc):
    bag = {}
    for word in doc.split():
        bag[word] = bag.get(word, 0) + 1
    return bag

## Implementing Naive Bayes

Now, implement a master function to build a naive Bayes classifier. Be sure to use the logarithmic probabilities to avoid underflow.

In [80]:
# Your code here
def classify_doc(doc, class_word_freq, V, return_posteriors=False):
    bag = bag_it(doc)
    classes = []
    posteriors = []
    for class_ in class_word_freq.keys():
        p = np.log(0.5)
        for word in bag.keys():
            num = bag[word]+1
            denom = class_word_freq[class_].get(word, 0) + V
            p += np.log(num/denom)
        classes.append(class_)
        posteriors.append(p)
    if return_posteriors:
        print(posteriors)
    return classes[np.argmax(posteriors)]

In [90]:
def classify_doc2(doc, class_word_freq, V, return_posteriors=False):
    bag = bag_it(doc)
    classes = []
    posteriors = []
    for class_ in class_word_freq.keys():
        p = np.log(0.5)
        for word in bag.keys():
            num = bag[word]+1
            denom = class_word_freq[class_].get(word, 0) + V
            p += np.log(num/denom)
        classes.append(class_)
        posteriors.append(p)
    if return_posteriors:
        print(posteriors)
    return classes[np.argmax(posteriors)]

In [85]:
np.argmax(posteriors_list)

1

In [71]:
train_df.iloc[0]

label                                                  ham
text     Lol I was gonna last month. I cashed some in b...
Name: 3110, dtype: object

In [67]:
train_df.head()

,label,text
3110,ham,Lol I was gonna last month. I cashed some in b...
312,spam,Think ur smart ? Win £200 this week in our wee...
844,spam,Urgent! call 09066350750 from your landline. Y...
1821,ham,I'll probably be by tomorrow (or even later to...
909,ham,WHITE FUDGE OREOS ARE IN STORES


## Test your classifier

Finally, test your classifier and measure its accuracy. Don't be perturbed if your results are sub-par; industry use cases would require substantial additional preprocessing before implementing the algorithm in practice.

In [91]:
y_hat_train = X_train['text'].map(lambda x: classify_doc2(x, class_word_freq, V))
residuals = y_train == y_hat_train
residuals.value_counts(normalize=True)

False    0.765179
True     0.234821
dtype: float64

In [60]:
y_hat_test = X_test['text'].map(lambda x: classify_doc(x, class_word_freq, V))
residuals = y_test == y_hat_test
residuals.value_counts(normalize=True)

False    0.735294
True     0.264706
dtype: float64

## Level up (Optional)

Rework your code into an appropriate class structure so that you could easily implement the algorithm on any given dataset.

## Summary

Well done! In this lab, you practiced implementing Naive Bayes for document classification!